In [13]:

import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from keras_tuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np
import random

In [2]:
circuits_file = open("circuits.csv")
circuits = list(list())
for i in circuits_file.readlines():
    circuits.append(i.strip().split(','))
races_file = open("races.csv")
races = list(list())
for i in races_file.readlines():
    races.append(i.strip().split(','))
qualifying_file = open("qualifying.csv")
quals = list(list())
for i in qualifying_file.readlines():
    quals.append(i.strip().split(','))
pit_stops_file = open("pit_stops.csv")
pit_stops = list(list())
for i in pit_stops_file.readlines():
    pit_stops.append(i.strip().split(','))
pit_stops_results = list(list())
for i in pit_stops:
    if i[2] != "1":
        for j in range(len(pit_stops_results)):
            if pit_stops_results[j][0] == int(i[0]) and pit_stops_results[j][1] == int(i[1]):
                pit_stops_results[j][2] += 1
    else:
        pit_stops_results.append([np.int(i[0]), np.int(i[1]), 1])
parameters = list(list())
for i in range(len(pit_stops_results)):
    parameters.append(list())
    parameters[i].append((np.int(races[pit_stops_results[i][0]][1]) - 1950) * 0.01)
    parameters[i].append((np.float(circuits[int(races[pit_stops_results[i][0]][3])][5]) + 38) * 0.01)
    parameters[i].append((np.float(circuits[int(races[pit_stops_results[i][0]][3])][6]) + 120) * 0.001)
    parameters[i].append((np.float(circuits[int(races[pit_stops_results[i][0]][3])][7]) + 8) * 0.0001)
    for j in range(len(quals)):
        if j != 0 and int(quals[j][1]) == pit_stops_results[i][0] and int(quals[j][2]) == pit_stops_results[i][1]:
            parameters[i].append(np.int(quals[j][5]) * 0.01)
            break
    if len(parameters[i]) < 5:
        parameters[i].append(np.int((random.random() * 100) % 20))


for i in range(len(pit_stops_results)):
    del pit_stops_results[i][0:2]
    
parameters_train = parameters[:(len(parameters) // 7 * 6)]

for i in range(len(parameters_train)):
    parameters_train[i] = tf.convert_to_tensor(parameters_train[i], dtype=tf.float32)
    
parameters_test = parameters[(len(parameters) // 7 * 6):]

for i in range(len(parameters_test)):
    parameters_test[i] = tf.convert_to_tensor(parameters_test[i], dtype=tf.float32)

pit_stops_train = pit_stops_results[:(len(pit_stops_results) // 7 * 6)]

for i in range(len(pit_stops_train)):
    pit_stops_train[i] = tf.convert_to_tensor(pit_stops_train[i], dtype=tf.float32)
    
pit_stops_test = pit_stops_results[(len(pit_stops_results) // 7 * 6):]

for i in range(len(pit_stops_test)):
    pit_stops_test[i] = tf.convert_to_tensor(pit_stops_test[i], dtype=tf.float32)

In [3]:


def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])    
    model.add(Dense(units=hp.Int('units_input',    
                                   min_value=512,    
                                   max_value=1024,   
                                   step=32),
                    input_dim=5,
                    activation=activation_choice))
    model.add(Dense(units=hp.Int('units_hidden',        
                                   min_value=128,   
                                   max_value=600,   
                                   step=32),
                    activation=activation_choice))   
    model.add(Dense(1, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,                
    objective='val_accuracy', 
                              
    max_trials=400,             
    directory='test_directory'   
    )

INFO:tensorflow:Reloading Oracle from existing project test_directory\untitled_project\oracle.json


In [4]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh', 'elu', 'selu'], 'ordered': False}
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': None}
units_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 600, 'step': 32, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'SGD'], 'ordered': False}


In [5]:
parameters_train = tf.stack(parameters_train)
pit_stops_train = tf.stack(pit_stops_train)
tuner.search(parameters_train,             
             pit_stops_train,               
             batch_size=256,           
             epochs=30,                
             validation_split=0.2,     
             )


Trial 400 Complete [00h 00m 02s]
val_accuracy: 0.6205882430076599

Best val_accuracy So Far: 0.6205882430076599
Total elapsed time: 00h 13m 26s
INFO:tensorflow:Oracle triggered exit


In [6]:
tuner.results_summary()

Results summary
Results in test_directory\untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
activation: relu
units_input: 864
units_hidden: 352
optimizer: adam
Score: 0.6205882430076599
Trial summary
Hyperparameters:
activation: tanh
units_input: 544
units_hidden: 192
optimizer: adam
Score: 0.6205882430076599
Trial summary
Hyperparameters:
activation: selu
units_input: 896
units_hidden: 480
optimizer: adam
Score: 0.6205882430076599
Trial summary
Hyperparameters:
activation: tanh
units_input: 544
units_hidden: 288
optimizer: rmsprop
Score: 0.6205882430076599
Trial summary
Hyperparameters:
activation: elu
units_input: 832
units_hidden: 544
optimizer: rmsprop
Score: 0.6205882430076599
Trial summary
Hyperparameters:
activation: sigmoid
units_input: 928
units_hidden: 352
optimizer: adam
Score: 0.6205882430076599
Trial summary
Hyperparameters:
activation: sigmoid
units_input: 928
units_hidden: 192
optimizer: adam
Score: 0.6

In [17]:
models = tuner.get_best_models(num_models=3)
main_model = models[0]
main_model.save('main_model.h5')

In [14]:
parameters_test = tf.stack(parameters_test)
pit_stops_test = tf.stack(pit_stops_test)
for model in models:
  model.summary()
  model.evaluate(parameters_test, pit_stops_test)
  print() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 864)               5184      
_________________________________________________________________
dense_1 (Dense)              (None, 352)               304480    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 353       
Total params: 310,017
Trainable params: 310,017
Non-trainable params: 0
_________________________________________________________________
18/18 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4038

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 544)               3264      
_________________________________________________________________
dens

C:\Users\Danila\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


18/18 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4038

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 896)               5376      
_________________________________________________________________
dense_1 (Dense)              (None, 480)               430560    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 481       
Total params: 436,417
Trainable params: 436,417
Non-trainable params: 0
_________________________________________________________________
18/18 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4038

